## SQL Database for faostat

### Import Dependencies

In [76]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import text

IUCN data is not in the database due to Red List Terms and Conditions stating no reposting: https://www.iucnredlist.org/es/terms/terms-of-use#:~:text=You%20may%20not%20repost%2C%20or,web%20maps%20that%20grant%20users

In [77]:
# Define the destination folder and database path
destination_folder = 'data'
database_filename = 'faostat_data.sqlite'
destination_path = os.path.join(destination_folder, database_filename)

# Create an SQLite engine
engine = create_engine(f'sqlite:///{destination_path}')

In [78]:
# Load CSVs into DataFrames
qcl = pd.read_csv('data/faostat/QCL_clean.csv')
rl = pd.read_csv('data/faostat/RL_clean.csv')

# Check the DataFrame
print(qcl.head())
print(rl.head())

          Area  Item  Year Unit   Value
0  Afghanistan  Rice  1961   ha  210000
1  Afghanistan  Rice  1962   ha  210000
2  Afghanistan  Rice  1963   ha  210000
3  Afghanistan  Rice  1964   ha  220000
4  Afghanistan  Rice  1965   ha  220000
          Area  Year  Land area  Cropland  Permanent meadows and pastures  \
0  Afghanistan  1961    65223.0    7750.0                         30000.0   
1  Afghanistan  1962    65223.0    7800.0                         30000.0   
2  Afghanistan  1963    65223.0    7850.0                         30000.0   
3  Afghanistan  1964    65223.0    7905.0                         30000.0   
4  Afghanistan  1965    65223.0    7910.0                         30000.0   

   Forest land  Other land  Farm buildings and Farmyards  Unaccounted  
0          0.0         0.0                           0.0      27473.0  
1          0.0         0.0                           0.0      27423.0  
2          0.0         0.0                           0.0      27373.0  
3        

In [79]:
# Load qcl DataFrame
qcl.to_sql('qcl', con=engine, if_exists='replace', index=False)

# Load rl DataFrame
rl.to_sql('rl', con=engine, if_exists='replace', index=False)

# Verify tables
qcl_from_db = pd.read_sql('SELECT * FROM qcl', con=engine)
rl_from_db = pd.read_sql('SELECT * FROM rl', con=engine)

# Check the data loaded
print(qcl_from_db.head())
print(rl_from_db.head())

          Area  Item  Year Unit   Value
0  Afghanistan  Rice  1961   ha  210000
1  Afghanistan  Rice  1962   ha  210000
2  Afghanistan  Rice  1963   ha  210000
3  Afghanistan  Rice  1964   ha  220000
4  Afghanistan  Rice  1965   ha  220000
          Area  Year  Land area  Cropland  Permanent meadows and pastures  \
0  Afghanistan  1961    65223.0    7750.0                         30000.0   
1  Afghanistan  1962    65223.0    7800.0                         30000.0   
2  Afghanistan  1963    65223.0    7850.0                         30000.0   
3  Afghanistan  1964    65223.0    7905.0                         30000.0   
4  Afghanistan  1965    65223.0    7910.0                         30000.0   

   Forest land  Other land  Farm buildings and Farmyards  Unaccounted  
0          0.0         0.0                           0.0      27473.0  
1          0.0         0.0                           0.0      27423.0  
2          0.0         0.0                           0.0      27373.0  
3        

### Basic Analysis

In [80]:
# Sum of each crop by country and year
query = """
SELECT Area, Item, Year, SUM(Value) AS Total_Production
FROM qcl
GROUP BY Area, Item, Year
ORDER BY Area, Year, Total_Production DESC
"""

crop_production_by_year = pd.read_sql(query, con=engine)

print(crop_production_by_year)

              Area                  Item  Year  Total_Production
0      Afghanistan  Cattle and Buffaloes  1961           2900000
1      Afghanistan      Green corn/Maize  1961            500000
2      Afghanistan                  Rice  1961            210000
3      Afghanistan  Cattle and Buffaloes  1962           3200000
4      Afghanistan      Green corn/Maize  1962            500000
...            ...                   ...   ...               ...
45429     Zimbabwe  Cattle and Buffaloes  2022           5438771
45430     Zimbabwe      Green corn/Maize  2022           1001819
45431     Zimbabwe            Soya beans  2022             38177
45432     Zimbabwe         Coffee, green  2022              2645
45433     Zimbabwe                  Rice  2022              2144

[45434 rows x 4 columns]


In [81]:
# Define SQL Query
query_percentages = """
SELECT
    Area,
    Year,
    100.0 AS Land_area_Percentage,
    (Cropland / [Land area]) * 100 AS Cropland_Percentage,
    ([Permanent meadows and pastures] / [Land area]) * 100 AS Permanent_meadows_and_pastures_Percentage,
    ([Forest land] / [Land area]) * 100 AS Forest_land_Percentage,
    ([Other land] / [Land area]) * 100 AS Other_land_Percentage,
    ([Farm buildings and Farmyards] / [Land area]) * 100 AS Farm_buildings_and_Farmyards_Percentage,
    ([Unaccounted] / [Land area]) * 100 AS Unaccounted_Percentage
FROM rl;
"""

percentages_df = pd.read_sql(query_percentages, con=engine)

print(percentages_df)

              Area  Year  Land_area_Percentage  Cropland_Percentage  \
0      Afghanistan  1961                 100.0            11.882311   
1      Afghanistan  1962                 100.0            11.958972   
2      Afghanistan  1963                 100.0            12.035632   
3      Afghanistan  1964                 100.0            12.119958   
4      Afghanistan  1965                 100.0            12.127624   
...            ...   ...                   ...                  ...   
13313     Zimbabwe  2018                 100.0             8.705261   
13314     Zimbabwe  2019                 100.0             8.240085   
13315     Zimbabwe  2020                 100.0             8.475800   
13316     Zimbabwe  2021                 100.0             8.107633   
13317     Zimbabwe  2022                 100.0             8.211011   

       Permanent_meadows_and_pastures_Percentage  Forest_land_Percentage  \
0                                      45.996044                0.00000

In [82]:
# Define SQL query
query = """
SELECT 
    Area,
    SUM(Cropland) AS Total_Cropland,
    SUM("Land area") AS Total_Land_Area,
    (SUM(Cropland) / SUM("Land area")) * 100 AS Cropland_Percentage
FROM 
    rl
WHERE 
    Year = 2022
GROUP BY 
    Area
ORDER BY 
    Cropland_Percentage DESC;
"""

cropland_2022_percentage = pd.read_sql(query, con=engine)

print(cropland_2022_percentage)

                                 Area  Total_Cropland  Total_Land_Area  \
0                          Bangladesh       8818.0000          13017.0   
1                              Rwanda       1618.4000           2467.0   
2                             Burundi       1643.5085           2568.0   
3                             Comoros        118.0000            186.1   
4                              Tuvalu          1.8000              3.0   
..                                ...             ...              ...   
232       Falkland Islands (Malvinas)          0.0000           1217.0   
233                           Curaçao          0.0000             44.4   
234                  China, Macao SAR          0.0000              3.3   
235  Bonaire, Sint Eustatius and Saba          0.0000             32.2   
236                          Anguilla          0.0000              9.0   

     Cropland_Percentage  
0              67.742183  
1              65.601946  
2              63.999552  
3  

### Crop land use percentages by country for 2022

In [83]:
# Query to drop the table if it exists
query_drop = "DROP TABLE IF EXISTS rl_modified;"

# Execute
with engine.connect() as conn:
    conn.execute(text(query_drop))

# Create the modified table
query_multiply = """
CREATE TABLE rl_modified AS
SELECT
    Area,
    Year,
    "Land area" * 1000 AS Land_area,
    Cropland * 1000 AS Cropland,
    "Permanent meadows and pastures" * 1000 AS Permanent_meadows_and_pastures,
    "Forest land" * 1000 AS Forest_land,
    "Other land" * 1000 AS Other_land,
    "Farm buildings and Farmyards" * 1000 AS Farm_buildings_and_Farmyards,
    "Unaccounted" * 1000 AS Unaccounted
FROM rl;
"""

# Execute
with engine.connect() as conn:
    conn.execute(text(query_multiply))

In [84]:
# Define SQL query
query_percentage = """
WITH Agricultural_Land AS (
    SELECT
        Area,
        SUM(Cropland + "Permanent meadows and pastures" + "Farm buildings and Farmyards") AS Total_Agricultural_Land
    FROM rl_modified
    WHERE Year = 2022
    GROUP BY Area
),
Crop_Percentage AS (
    SELECT
        qcl.Area,
        qcl.Item,
        qcl.Year,
        al.Total_Agricultural_Land,
        qcl.Value AS Crop_Value,
        (qcl.Value / al.Total_Agricultural_Land) * 100 AS Agricultural_Land_Percentage
    FROM qcl
    JOIN Agricultural_Land al ON qcl.Area = al.Area
    WHERE qcl.Year = 2022
)
SELECT * FROM Crop_Percentage;
"""

crop_percentage_2022 = pd.read_sql(query_percentage, con=engine)

print(crop_percentage_2022)

                            Area                  Item  Year  \
0                    Afghanistan                  Rice  2022   
1                    Afghanistan  Cattle and Buffaloes  2022   
2                        Albania                  Rice  2022   
3                        Albania            Soya beans  2022   
4                        Albania  Cattle and Buffaloes  2022   
..                           ...                   ...   ...   
807                      Ukraine      Green corn/Maize  2022   
808  United Republic of Tanzania      Green corn/Maize  2022   
809     United States of America      Green corn/Maize  2022   
810                      Uruguay      Green corn/Maize  2022   
811                     Zimbabwe      Green corn/Maize  2022   

     Total_Agricultural_Land  Crop_Value  Agricultural_Land_Percentage  
0                  8051000.0      128000                      1.589865  
1                  8051000.0     5568618                     69.166787  
2           